In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Задача 2 (Циферки)  
от 0.98 - 10 баллов   
от 0.95 до 0.98 - 8 баллов  
от 0.93 до 0.95 - 6 баллов 

In [ ]:
train_dataset = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_dataset = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
def get_numpy_data(dataset):
    labels = dataset.get('label', pd.DataFrame()).to_numpy()
    images = dataset.loc[:, dataset.columns != 'label'].to_numpy().astype("float32") / 255
    return images, labels

In [ ]:
images, labels = get_numpy_data(train_dataset) 
test_images, _ = get_numpy_data(test_dataset)

val_size = int(images.shape[0] * 0.1)
train_images, val_images = images[val_size:], images[:val_size]
train_labels, val_labels = labels[val_size:], labels[:val_size]

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.3)(features)
    features = layers.Dense(256, activation="relu")(features)
    features = layers.Dropout(0.2)(features)
    features = layers.Dense(128, activation="relu")(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
def callbacks(name): 
    return [ 
        EarlyStopping(monitor = 'val_loss', patience = 6), 
        ReduceLROnPlateau(monitor = 'val_loss', patience = 3), 
    ]

In [ ]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])
history_model = model.fit(
    train_images, train_labels,
    epochs=40,
    batch_size=1024,
    callbacks = callbacks('mlp_reg'),
    validation_data=(val_images, val_labels))

In [ ]:
len(history_model.history["val_loss"])

In [ ]:

import matplotlib.pyplot as plt
val_loss = history_model.history["val_loss"]
epochs = range(1, len(val_loss) + 1)
plt.plot(epochs, val_loss, "b--",
         label="Validation loss")
plt.title("Effect of insufficient model capacity on validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

In [ ]:
score = model.evaluate(val_images, val_labels, verbose = 0)
print('Test loss: {}%'.format(score[0] * 100))
print('Test score: {}%'.format(score[1] * 100))

print("MLP Error: %.2f%%" % (100 - score[1] * 100))

In [ ]:
# test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

In [ ]:

def prediction_to_dataframe(predictions):
    result = pd.DataFrame({
            "Label": predictions.argmax(axis=1)
        })
    result.index += 1
    result.index.name = "ImageId"
    return result

def make_submission(predictions_dataframe):
    path="/kaggle/working/submission.csv"
    predictions_dataframe.to_csv(path)
    print(f"Submission exported to {path}")
    
predictions_dataframe = prediction_to_dataframe(predictions)
make_submission(predictions_dataframe)

